In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import matplotlib.pyplot as plt

# Quick processing: rocklin to FASTA
data from: https://github.com/duerrsimon/mars-biohackathon
* [Multi-mutant](https://github.com/duerrsimon/mars-biohackathon/blob/main/data/multiple_muts_train.csv)
* [Single-mutant](https://github.com/duerrsimon/mars-biohackathon/blob/main/data/single_muts_train.csv)

In [11]:
df = pd.read_csv("../rocklin_single_mutant.csv")
df[df.stabilityscore > 0.].shape

(8521, 5)

In [8]:
df2 = pd.read_csv("../rocklin_multi_mutant.csv")
df2.head()

,Unnamed: 0,name,sequence,secondary_structure,stabilityscore
0,0,EHEE_rd1_0392.pdb,GSSTETYEVHTTDEAERVRKELEKKGYKVEVRSDGTEYEIRSE,TTTTEEEEETTHHHHHHHHHHHHHHTTTEEEEEETTEEEEEET,-0.32
1,1,EHEE_rd1_0925.pdb_random,GSSEKEDVHRFEIATWERDKQLEERTGREGRTRDIANREDERE,TTTTEEEEETTHHHHHHHHHHHHHHTTTEEEEEETTEEEEEET,-0.50
2,2,HEEH_rd1_0699.pdb_hp,EARKSDGPSGVKRAKKNEDKETDRKTALYNKIQRVAKEITENE,THHHHHHHHHHHHHTTEEEETTEEEETTTHHHHHHHHHHHHHT,-0.33
3,3,EEHEE_rd1_0108.pdb_hp,GSAQFKNGNEESEVPYKGRRVKQEPKKDANEWERQGTEGRSDG,TEEEETTEEEETTTHHHHHHHHHHHHHHHTTTEEEETTEEEET,-0.18
4,4,EEHEE_rd1_0483.pdb,GSGTVELNGNRYTSNSEEDIREWIRRQGATTSETSNGKYQVHR,TEEEETTEEEETTTHHHHHHHHHHHHHHHTTTEEEETTEEEET,-0.12


Let's use the multi mutant dataset and filter out for those with a stability score > 0.1:

In [17]:
print(df2.shape)
print(df2[df2.stabilityscore > 0.1].shape)

(29187, 5)
(9903, 5)


In [37]:
df3 = df2[df2.stabilityscore > 0.1] 
print(df3.shape)
df3 = df3.drop_duplicates(subset="sequence", keep="first")
print(df3.shape)

(9903, 5)
(9880, 5)


In [38]:
# df3.to_csv("../rocklin_multi_mutant_stable.csv")

In [41]:
df3.sequence.str.len().max()

43

In [42]:
df3.sequence

28       GSRSGNFGGKSKSPTDWTVKTTATKYAKADTTKDEYGKERLDT
30       GSADLTVGNKQFDGADDENQKRARETAGGTHVERDGNTVQVKK
33       SDSKEEARRRAKKKNTKVTVNGVTVRAETEEAEKRAEEWLRRQ
37       SEEERERAREKARRGEPIKVNGEEWEKPSTELAERLADKKANG
41       GSGESRFKGQRVETDNPKELAEREARKVNGPFTQTNGRVEVQV
                            ...                     
29176    GTDESTNYTYTAPSTDAARKFAQELAKQLGLKVTEKNGTFKVT
29178    TWEAAQRVLQALKDNKPITINGVTFTPGNPEVKKWDKKVANAK
29181    GSHTLKVGNKEYTFPSWKEAEKFAKKVGAKKVTVKNGTVEVHQ
29184    PAAQKAADELKKGATNVTVTLNGKKITLTATSPEEAEKAVKKF
29186    TKVHVNGKTYTFPSPDLAKKFAEKVAKKYNLTVHVDGNTVKVK
Name: sequence, Length: 9880, dtype: object

In [43]:
from plaid.utils import write_to_fasta

In [45]:
# write_to_fasta(df3.sequence.values, "rocklin_stable.fasta", df3.name.values)

Wrote 9880 sequences to rocklin_stable.fasta.


# Examine perplexity of rocklin dataset

In [46]:
from plaid.evaluation import RITAPerplexity
import torch

device = "cuda"

perplexity_calculator = RITAPerplexity(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [50]:
sequences = list(df3.sequence.values)
perplexity_calculator.batch_eval(sequences, batch_size=256)

15.40421669299786

# Structure

In [3]:
sequences_df = pd.read_csv("/homefs/home/lux70/storage/data/rocklin/rocklin_multi_mutant_stable.csv")
print(sequences_df.shape)
sequences_df.head()

(9880, 6)


,Unnamed: 0.1,Unnamed: 0,name,sequence,secondary_structure,stabilityscore
0,28,28,EEHEE_rd1_0657.pdb_hp,GSRSGNFGGKSKSPTDWTVKTTATKYAKADTTKDEYGKERLDT,TEEEETTEEEETTTHHHHHHHHHHHHHHHTTTEEEETTEEEET,0.43
1,30,30,EEHEE_rd1_0070.pdb,GSADLTVGNKQFDGADDENQKRARETAGGTHVERDGNTVQVKK,TEEEETTEEEETTTHHHHHHHHHHHHHHHTTTEEEETTEEEET,0.44
2,33,33,HEEH_rd1_0524.pdb,SDSKEEARRRAKKKNTKVTVNGVTVRAETEEAEKRAEEWLRRQ,THHHHHHHHHHHHHTTEEEETTEEEETTTHHHHHHHHHHHHHT,0.43
3,37,37,HEEH_rd1_0724.pdb,SEEERERAREKARRGEPIKVNGEEWEKPSTELAERLADKKANG,THHHHHHHHHHHHHTTEEEETTEEEETTTHHHHHHHHHHHHHT,0.45
4,41,41,EEHEE_rd1_0095.pdb,GSGESRFKGQRVETDNPKELAEREARKVNGPFTQTNGRVEVQV,TEEEETTEEEETTTHHHHHHHHHHHHHHHTTTEEEETTEEEET,0.21


In [4]:
import os
struct_files_dir = "/homefs/home/lux70/storage/data/rocklin/structures/"
structure_files = os.listdir(struct_files_dir)
print(len(structure_files))

16463


In [5]:
print(len(sequences_df.name.unique()))
print(len(sequences_df.name.map(lambda x: x.split(".")[0]).unique()))

9880
6989


In [6]:
len(set(sequences_df.name).intersection(set(structure_files)))

4731

In [7]:
files_to_examine = list(set(sequences_df.name).intersection(set(structure_files)))
print(files_to_examine[:10])

['EEHEE_rd3_0703.pdb', 'HHH_rd2_0070.pdb', 'EEHEE_rd3_0660.pdb', 'HHH_rd1_0753.pdb', 'EHEE_rd2_0969.pdb', 'HHH_rd3_0147.pdb', 'HHH_rd3_0180.pdb', 'EEHEE_rd2_0709.pdb', 'HHH_rd1_0090.pdb', 'HEEH_rd3_1466.pdb']


In [13]:
from plaid.utils import view_py3Dmol

for i in range(4):
    fpath = struct_files_dir + files_to_examine[i]
    with open(fpath, "r") as f:
        pdbstr = f.read()
    view_py3Dmol(pdbstr)

ModuleNotFoundError: No module named 'plaid'

In [32]:
predicted_pdb_strs_dir = "/homefs/home/lux70/storage/data/rocklin/esmfold_predicted_structures/val/" 
print(len(os.listdir(predicted_pdb_strs_dir)))
print(len(set(os.listdir(predicted_pdb_strs_dir)).intersection(set(files_to_examine))))

1976
932


In [33]:
files_to_examine = list(set(os.listdir(predicted_pdb_strs_dir)).intersection(set(files_to_examine)))
from plaid.utils import run_tmalign

scores = []
for filename in files_to_examine:
    scores.append(run_tmalign(f"{struct_files_dir}{filename}", f"{predicted_pdb_strs_dir}{filename}")) 




FileNotFoundError: TMalign not found in PATH